In [11]:
import json
import numpy as np
import logging
import os
import math
import copy
import random
import string
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple, Set

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- File Paths ---
# !!! PLEASE UPDATE THESE PATHS TO YOUR ACTUAL FILE LOCATIONS !!!
RELAY_PAIRS_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json"
OPTIMIZED_SETTINGS_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values_all_relays.json" # Updated output name

# --- Constants ---
K = 0.14
N = 0.02
CTI = 0.2
# Optimization Bounds and Parameters
MIN_TDS = 0.05 # Lower practical limit for TDS
MAX_TDS = 1.0 # Max TDS limit (changed from 1 to 1.0 for clarity)
MIN_PICKUP = 0.05
# Stricter pickup limit relative to Ishc, inspired by snippet
MAX_PICKUP_FACTOR = 0.7
# MAX_TIME still needed for constraint violations, but not for calculation errors
MAX_TIME = 20.0
MAX_ITERATIONS = 250 # More iterations might be needed
# Convergence Targets
TARGET_TMT = -0.005 # Stricter TMT target again
CONVERGENCE_THRESHOLD_TMT = 0.005 # Smaller margin for TMT
CONVERGENCE_THRESHOLD_MT = -0.01 # Max individual MT allowed
# Objective Function Weights
W_TIME = 0.1 # Further decrease weight of total time
W_MT = 15.0 # Increase weight for miscoordination penalty
W_PICKUP_DIFF = 0.0 # Keep as 0 if not used
# Adjustment Step Sizes & Factors (Inspired by Snippet)
AGGRESSIVE_MT_THRESHOLD = -CTI * 0.75 # Threshold to trigger aggressive adjustments
# Aggressive Steps
AGGRESSIVE_TDS_BACKUP_FACTOR = 1.15 # Increase backup TDS more
AGGRESSIVE_TDS_MAIN_FACTOR = 0.90  # Decrease main TDS more
AGGRESSIVE_PICKUP_BACKUP_FACTOR = 1.05 # Increase backup pickup if TDS maxed out
# Normal Steps (additive for TDS, small mult for pickup)
NORMAL_TDS_BACKUP_ADD = 0.02
NORMAL_TDS_MAIN_SUB = 0.01
NORMAL_PICKUP_BACKUP_FACTOR = 1.01 # Keep small adjustments
NORMAL_PICKUP_MAIN_FACTOR = 0.99 # Keep small adjustments


# --- Helper Functions ---
def load_json_file(file_path: str) -> Optional[Any]:
    try:
        with open(file_path, 'r') as file: data = json.load(file)
        logger.info(f"Archivo cargado: {file_path}"); return data
    except FileNotFoundError: logger.error(f"No encontrado: {file_path}"); return None
    except json.JSONDecodeError as e: logger.error(f"JSON inválido: {file_path}: {e}"); return None
    except Exception as e: logger.error(f"Error carga {file_path}: {e}"); return None

def calculate_operation_time(I_shc: float, I_pi: float, TDS: float) -> Optional[float]:
    """Calculates relay operation time. Returns None if calculation is impossible."""
    if I_pi <= 0 or I_shc <= 0 or TDS < MIN_TDS or TDS > MAX_TDS:
        return None
    if I_pi > I_shc * MAX_PICKUP_FACTOR:
        # This is a constraint VIOLATION, return MAX_TIME to penalize
        return MAX_TIME
    M = I_shc / I_pi
    if M <= 1.0001: # Ratio too low for standard curve calculation
        # This indicates a calculation impossibility or extremely long time
        return None # Return None if calculation cannot proceed normally
    try:
        denominator = M**N - 1
        if abs(denominator) < 1e-9: # Avoid division by zero
            return None
        time = TDS * (K / denominator)
        if not np.isfinite(time) or time <= 0:
             return None
        # Return calculated time, capped by MAX_TIME (as a practical upper limit if calculation is valid)
        # If pickup constraint caused MAX_TIME earlier, this min() won't change it.
        return min(time, MAX_TIME)
    except (OverflowError, ValueError) as e:
        return None
    except Exception as e:
        logger.error(f"Excepción inesperada en calc_op_time (I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}): {e}")
        return None

def group_data_by_scenario(relay_pairs_data: List[Dict]) -> Dict[str, Dict[str, Any]]:
    scenario_map: Dict[str, Dict[str, Any]] = {}
    processed_pairs_count = 0
    skipped_pairs_count = 0
    for i, pair_entry in enumerate(relay_pairs_data):
        scenario_id = pair_entry.get("scenario_id")
        main_relay_info = pair_entry.get('main_relay'); backup_relay_info = pair_entry.get('backup_relay')
        if not scenario_id or not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            skipped_pairs_count+=1; continue
        if scenario_id not in scenario_map: scenario_map[scenario_id] = {"pairs_info": [], "initial_settings": {}, "relays": set()}
        main_relay = main_relay_info.get('relay'); backup_relay = backup_relay_info.get('relay')
        I_shc_main = main_relay_info.get('Ishc'); I_shc_backup = backup_relay_info.get('Ishc')
        if not (main_relay and backup_relay and
                isinstance(I_shc_main, (int, float)) and I_shc_main > 0 and
                isinstance(I_shc_backup, (int, float)) and I_shc_backup > 0):
            skipped_pairs_count+=1; continue

        processed_pairs_count += 1
        scenario_map[scenario_id]['relays'].add(main_relay); scenario_map[scenario_id]['relays'].add(backup_relay)
        initial_settings_scenario = scenario_map[scenario_id]['initial_settings']
        for r_name, r_info in [(main_relay, main_relay_info), (backup_relay, backup_relay_info)]:
            if r_name not in initial_settings_scenario:
                tds = r_info.get('TDS'); pickup = r_info.get('pick_up')
                if isinstance(tds, (int, float)) and isinstance(pickup, (int, float)):
                    initial_settings_scenario[r_name] = {
                        'TDS_initial': float(tds),
                        'pickup_initial': float(pickup)
                         }
        scenario_map[scenario_id]['pairs_info'].append({"main_relay": main_relay, "backup_relay": backup_relay, "I_shc_main": float(I_shc_main), "I_shc_backup": float(I_shc_backup)})

    logger.info(f"Datos agrupados por escenario. Pares procesados: {processed_pairs_count}, Pares omitidos: {skipped_pairs_count}")
    return scenario_map


# --- Scenario-Specific Optimization Function (MODIFIED for None handling) ---
def run_scenario_optimization(
    scenario_id: str,
    pairs_info: List[Dict],
    initial_settings: Dict[str, Dict[str, float]],
    relays_in_scenario: Set[str]
) -> Dict[str, Dict[str, Optional[float]]]: # Return type allows None for TDS/pickup
    """Performs iterative optimization, handles calculation errors (None), excludes errors from TMT."""

    logger.info(f"--- Iniciando optimización para {scenario_id} ---")
    if not pairs_info:
        logger.warning(f"({scenario_id}) No hay pares válidos para optimizar.")
        # Return empty dict, final formatting will handle missing relays
        return {}

    # 1. Initialize settings - Use initial Pickup, MIN TDS
    relay_settings: Dict[str, Dict[str, float]] = {} # Store current TDS/pickup floats
    default_pickup = MIN_PICKUP * 1.5 # Start slightly above min if no initial found

    for relay in relays_in_scenario:
         # Use max(MIN_PICKUP, ...) to ensure pickup >= lower bound initially
         initial_pickup = default_pickup
         if relay in initial_settings and 'pickup_initial' in initial_settings[relay]:
             initial_pickup = max(MIN_PICKUP, initial_settings[relay]['pickup_initial'])
         else:
             logger.warning(f"({scenario_id}) Config inicial (pickup) no encontrada o inválida para '{relay}'. Usando defecto ajustado: {initial_pickup:.4f}")

         relay_settings[relay] = {
             "TDS": MIN_TDS, # Start TDS at minimum
             "pickup": initial_pickup
         }

    # --- Optimization Loop ---
    last_tmt = float('inf')
    no_improvement_streak = 0

    for iteration in range(MAX_ITERATIONS):
        total_main_time = 0.0; tmt = 0.0; miscoordination_penalty = 0.0
        current_pair_results = []; max_neg_mt = 0
        error_pairs_count = 0 # Count pairs with calculation errors

        # Calculate performance
        for pair in pairs_info:
            main_relay = pair["main_relay"]; backup_relay = pair["backup_relay"]
            I_shc_main = pair["I_shc_main"]; I_shc_backup = pair["I_shc_backup"]
            # Check if relays (still) exist in settings - should always be true here
            if main_relay not in relay_settings or backup_relay not in relay_settings:
                logger.error(f"({scenario_id}) Iter {iteration+1}: Relay faltante en settings {main_relay} o {backup_relay}. Omitiendo par.")
                continue

            tds_main = relay_settings[main_relay]["TDS"]
            pickup_main = relay_settings[main_relay]["pickup"]
            tds_backup = relay_settings[backup_relay]["TDS"]
            pickup_backup = relay_settings[backup_relay]["pickup"]

            # Apply pickup lower bound BEFORE calculating time
            # MAX_PICKUP_FACTOR constraint is checked inside calculate_operation_time
            pickup_main_bounded = max(MIN_PICKUP, pickup_main)
            pickup_backup_bounded = max(MIN_PICKUP, pickup_backup)
            # Apply TDS bounds (redundant with init/adjustment, but safe)
            tds_main_bounded = min(MAX_TDS, max(MIN_TDS, tds_main))
            tds_backup_bounded = min(MAX_TDS, max(MIN_TDS, tds_backup))

            # Calculate times - MODIFIED: handle None return
            main_time = calculate_operation_time(I_shc_main, pickup_main_bounded, tds_main_bounded)
            backup_time = calculate_operation_time(I_shc_backup, pickup_backup_bounded, tds_backup_bounded)

            delta_t: Optional[float] = None; mt: Optional[float] = None

            if main_time is None or backup_time is None:
                delta_t = None; mt = None
                error_pairs_count += 1
            # Check for MAX_TIME (constraint violation or very long calc time) next
            elif main_time >= MAX_TIME or backup_time >= MAX_TIME:
                 # Treat as large miscoordination if either is MAX_TIME
                 # We can still calculate a delta/mt, but it will be large
                 # Use MAX_TIME for the non-operating one if needed
                 effective_main_time = MAX_TIME if main_time is None or main_time >= MAX_TIME else main_time
                 effective_backup_time = MAX_TIME if backup_time is None or backup_time >= MAX_TIME else backup_time
                 # Avoid large negative delta if main is MAX_TIME and backup is calculated
                 if effective_main_time >= MAX_TIME and effective_backup_time < MAX_TIME:
                     delta_t = -MAX_TIME # Penalize main relay not operating
                 else:
                     delta_t = effective_backup_time - effective_main_time # Could be near 0 if both MAX_TIME
                 mt = delta_t - CTI # Will likely be negative or large positive
            else: # Both times calculated and < MAX_TIME
                 delta_t = backup_time - main_time
                 mt = delta_t - CTI

            current_pair_results.append({
                "main_time": main_time, "backup_time": backup_time,
                "delta_t": delta_t, "mt": mt,
                "main_relay": main_relay, "backup_relay": backup_relay,
                "I_shc_main": I_shc_main, "I_shc_backup": I_shc_backup
            })

            # Accumulate performance metrics - Skip None values for TMT/Penalty
            if main_time is not None and main_time < MAX_TIME:
                total_main_time += main_time
            if mt is not None and mt < 0: # Only sum valid, negative MT values
                tmt += mt
                miscoordination_penalty += mt**2 # Penalize miscoordination quadratically
                max_neg_mt = min(max_neg_mt, mt)

        # Objective function calculation
        of = W_MT * miscoordination_penalty + W_TIME * total_main_time

        if (iteration + 1) % 25 == 0 or iteration == 0: # Log periodically
             logger.info(f"({scenario_id}) Iter {iteration+1}/{MAX_ITERATIONS}: OF={of:.4f}, TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Pairs_Errors={error_pairs_count}")

        # Check Convergence (based on valid mt values)
        # Allow convergence if TMT is slightly positive but close, AND individual MTs are fine
        if tmt >= TARGET_TMT and max_neg_mt >= CONVERGENCE_THRESHOLD_MT and abs(tmt) < CONVERGENCE_THRESHOLD_TMT :
             logger.info(f"({scenario_id}) Convergencia alcanzada en iteración {iteration+1} (TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}).")
             break

        # --- AJUSTE V3 (Skip pairs with mt=None) ---
        adjustments_made = False
        # Create a copy to modify for the *next* iteration
        next_relay_settings = copy.deepcopy(relay_settings)

        # Sort pairs by the most negative MT to prioritize fixing worst violations
        current_pair_results.sort(key=lambda p: p.get('mt', 0) if p.get('mt') is not None else 0)

        for pair_res in current_pair_results:
            mt_val = pair_res.get("mt")
            if mt_val is None: continue # Skip if MT couldn't be calculated

            # Adjust only significantly miscoordinated pairs (mt < threshold)
            if mt_val < CONVERGENCE_THRESHOLD_MT: # Adjust if MT is below the allowed negative margin
                main_relay = pair_res["main_relay"]; backup_relay = pair_res["backup_relay"]
                I_shc_main = pair_res["I_shc_main"]; I_shc_backup = pair_res["I_shc_backup"]

                # Get settings from the 'next_relay_settings' dict, as previous adjustments in this loop might affect them
                tds_main_curr = next_relay_settings[main_relay]["TDS"]
                tds_backup_curr = next_relay_settings[backup_relay]["TDS"]
                pickup_main_curr = next_relay_settings[main_relay]["pickup"]
                pickup_backup_curr = next_relay_settings[backup_relay]["pickup"]

                # Initialize new values with current ones
                new_tds_backup = tds_backup_curr; new_pickup_backup = pickup_backup_curr
                new_tds_main = tds_main_curr; new_pickup_main = pickup_main_curr

                # Determine adjustment strategy based on severity (mt_val vs AGGRESSIVE_MT_THRESHOLD)
                is_aggressive = mt_val < AGGRESSIVE_MT_THRESHOLD

                if is_aggressive:
                    # Aggressive: Increase backup TDS, decrease main TDS significantly
                    # logger.debug(f"({scenario_id}) Iter {iteration+1}: Ajuste Agresivo para {main_relay}/{backup_relay} (mt={mt_val:.4f})")
                    new_tds_backup = min(MAX_TDS, tds_backup_curr * AGGRESSIVE_TDS_BACKUP_FACTOR)
                    # If backup TDS hits max, slightly increase backup pickup (if possible)
                    if abs(new_tds_backup - MAX_TDS) < 1e-6:
                        max_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
                        new_pickup_backup = min(max_pickup_backup, max(MIN_PICKUP, pickup_backup_curr * AGGRESSIVE_PICKUP_BACKUP_FACTOR))
                    # Decrease main TDS
                    new_tds_main = max(MIN_TDS, tds_main_curr * AGGRESSIVE_TDS_MAIN_FACTOR)
                else:
                    # Normal: Small additive increase to backup TDS, subtractive for main TDS
                    # logger.debug(f"({scenario_id}) Iter {iteration+1}: Ajuste Normal para {main_relay}/{backup_relay} (mt={mt_val:.4f})")
                    new_tds_backup = min(MAX_TDS, max(MIN_TDS, tds_backup_curr + NORMAL_TDS_BACKUP_ADD))
                    new_tds_main = min(MAX_TDS, max(MIN_TDS, tds_main_curr - NORMAL_TDS_MAIN_SUB))
                    # Optional: Small pickup adjustments (consider if needed)
                    # max_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
                    # new_pickup_backup = min(max_pickup_backup, max(MIN_PICKUP, pickup_backup_curr * NORMAL_PICKUP_BACKUP_FACTOR))
                    # max_pickup_main = I_shc_main * MAX_PICKUP_FACTOR
                    # new_pickup_main = min(max_pickup_main, max(MIN_PICKUP, pickup_main_curr * NORMAL_PICKUP_MAIN_FACTOR))


                # Update the temporary settings dict for the *next* iteration
                # Ensure bounds are respected after calculation
                next_relay_settings[backup_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_backup))
                next_relay_settings[main_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_main))
                # Check max pickup based on relevant Ishc before setting
                max_pickup_backup_limit = I_shc_backup * MAX_PICKUP_FACTOR
                next_relay_settings[backup_relay]["pickup"] = min(max_pickup_backup_limit, max(MIN_PICKUP, new_pickup_backup))
                max_pickup_main_limit = I_shc_main * MAX_PICKUP_FACTOR
                next_relay_settings[main_relay]["pickup"] = min(max_pickup_main_limit, max(MIN_PICKUP, new_pickup_main))


                adjustments_made = True
        # --- End Adjustment ---

        if adjustments_made:
            relay_settings = next_relay_settings # Adopt the adjusted settings for the next iteration
        elif iteration > 15: # Stop if no adjustments needed after initial phase
             logger.info(f"({scenario_id}) No se realizaron ajustes en iteración {iteration + 1} y MTs están por encima del umbral. Deteniendo.")
             break

        # Check for stagnation (based on TMT from valid pairs)
        if abs(tmt - last_tmt) < 0.0001: # Very small threshold for TMT change
            no_improvement_streak += 1
        else:
            no_improvement_streak = 0
        last_tmt = tmt

        if no_improvement_streak >= 25: # Stop after 25 iterations with no TMT improvement
            logger.warning(f"({scenario_id}) TMT no mejora significativamente ({no_improvement_streak} iteraciones consecutivas). Deteniendo.")
            break

    else: # Executes if loop finishes without break (max iterations reached)
        logger.warning(f"({scenario_id}) La optimización no convergió después de {MAX_ITERATIONS} iteraciones.")

    # Format final results for relays optimized IN THIS SCENARIO
    # The final check for ALL relays happens outside this function
    formatted_settings: Dict[str, Dict[str, Optional[float]]] = {}
    relays_max_ishc = {} # Cache max Ishc per relay for final bounding

    # Find max Ishc seen by each relay *in this scenario*
    for relay in relays_in_scenario:
        max_ishc = 0
        for p in pairs_info:
             if p['main_relay'] == relay: max_ishc = max(max_ishc, p.get('I_shc_main', 0))
             if p['backup_relay'] == relay: max_ishc = max(max_ishc, p.get('I_shc_backup', 0))
        # If a relay was in relays_in_scenario but somehow had Ishc=0 (shouldn't happen with grouping logic), provide a fallback
        relays_max_ishc[relay] = max_ishc if max_ishc > 0 else 1.0 # Fallback Ishc=1.0 to avoid division by zero if default pickup is used

    # Apply final bounds and formatting
    for relay, settings in relay_settings.items():
        final_pickup = settings['pickup']
        final_tds = settings['TDS']
        max_ishc_relay = relays_max_ishc.get(relay, 1.0) # Use cached max Ishc

        # Final check: pickup <= Ishc * factor (using max Ishc seen by relay)
        final_pickup_bounded = min(max_ishc_relay * MAX_PICKUP_FACTOR, max(MIN_PICKUP, final_pickup))
        final_tds_bounded = min(MAX_TDS, max(MIN_TDS, final_tds))

        # Check if final settings allow calculation with max_ishc_relay
        # If calculate_operation_time returns None or MAX_TIME even with optimized settings and max Ishc,
        # it might indicate the relay cannot operate reliably under its most stressed condition.
        # We could potentially set to None here, but let's keep the calculated bounded values for now.
        # test_time = calculate_operation_time(max_ishc_relay, final_pickup_bounded, final_tds_bounded)
        # if test_time is None or test_time >= MAX_TIME:
        #     logger.warning(f"({scenario_id}) Relay {relay}: Final settings ({final_tds_bounded:.4f}, {final_pickup_bounded:.4f}) might not operate reliably with max Ishc ({max_ishc_relay:.2f}).")
            # Optionally set to None here if non-operation is critical
            # formatted_settings[relay] = {"TDS": None, "pickup": None}
            # continue # Skip adding the float values

        formatted_settings[relay] = {
            # Format to 5 decimal places as before
            "TDS": float(f"{final_tds_bounded:.5f}"),
            "pickup": float(f"{final_pickup_bounded:.5f}")
        }

    logger.info(f"--- Optimización finalizada para {scenario_id} (relés procesados: {len(formatted_settings)}) ---")
    # Return only the settings for relays processed in this scenario
    return formatted_settings


# --- Script Execution ---
if __name__ == "__main__":
    # Use a more descriptive log format if desired
    logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    logger.info("--- Iniciando Script de Optimización de Ajustes de Relés (v4 - Garantizar Todos los Relés) ---")

    # !!! ENSURE PATHS ARE CORRECT !!!
    if not os.path.exists(RELAY_PAIRS_PATH):
         logger.error(f"¡Error Crítico! No se encontró el archivo de entrada: {RELAY_PAIRS_PATH}")
         raise SystemExit("Archivo de pares de relés no encontrado.")

    relay_pairs_data = load_json_file(RELAY_PAIRS_PATH)
    if relay_pairs_data is None: raise SystemExit("Error crítico al cargar archivo de pares.")
    if not isinstance(relay_pairs_data, list):
        logger.error("El archivo de entrada no contiene una lista JSON válida.")
        raise SystemExit("Formato de entrada inválido.")

    # --- ADDED: Determine the master list of all relays ---
    all_relay_names: Set[str] = set()
    for pair_entry in relay_pairs_data:
        # Safely get relay names
        main_relay_info = pair_entry.get('main_relay')
        backup_relay_info = pair_entry.get('backup_relay')
        if isinstance(main_relay_info, dict):
            main_relay = main_relay_info.get('relay')
            if main_relay and isinstance(main_relay, str): all_relay_names.add(main_relay)
        if isinstance(backup_relay_info, dict):
            backup_relay = backup_relay_info.get('relay')
            if backup_relay and isinstance(backup_relay, str): all_relay_names.add(backup_relay)

    if not all_relay_names:
        logger.error("¡Error Crítico! No se encontraron nombres de relés válidos en el archivo de entrada.")
        raise SystemExit("No se pudieron identificar los relés del sistema.")
    logger.info(f"Lista maestra de relés creada. Total de relés únicos encontrados: {len(all_relay_names)}")
    # --- END ADDED SECTION ---


    scenario_data_map = group_data_by_scenario(relay_pairs_data)
    if not scenario_data_map:
        logger.error("Error crítico al procesar datos de entrada. No se encontraron escenarios o pares válidos para procesar.")
        # Decide whether to exit or proceed to write an empty file
        # raise SystemExit("Agrupación de escenarios fallida.")
        logger.warning("No se encontraron escenarios válidos para procesar.")


    all_optimized_settings_results = {} # Store results from run_scenario_optimization
    scenarios_processed_count = 0
    for scenario_id, scenario_data in scenario_data_map.items():
        if not scenario_data.get('relays') or not scenario_data.get('pairs_info'):
             logger.warning(f"Omitiendo escenario '{scenario_id}': No contiene relés o pares de información válidos después del procesamiento inicial.")
             continue

        # Run optimization, result contains settings only for relays in *this* scenario's pairs
        optimized_settings_for_scenario: Dict[str, Dict[str, Optional[float]]] = run_scenario_optimization(
            scenario_id,
            scenario_data['pairs_info'],
            scenario_data['initial_settings'],
            scenario_data['relays'] # Pass only relays involved in this scenario
        )
        # Store the potentially partial results; we'll fill gaps later
        all_optimized_settings_results[scenario_id] = optimized_settings_for_scenario
        scenarios_processed_count += 1


    # Format output as list - MODIFIED Section
    output_list = []
    if scenarios_processed_count > 0: # Check if any scenarios were actually processed
        logger.info(f"Formateando resultados para {scenarios_processed_count} escenarios procesados...")

        for scenario_id, optimized_settings in all_optimized_settings_results.items():
            current_timestamp = datetime.now(timezone.utc).isoformat(timespec='microseconds').replace('+00:00', 'Z')

            # --- MODIFIED: Ensure all relays from the master list are present ---
            final_relay_values_for_scenario: Dict[str, Optional[Dict[str, Optional[float]]]] = {}
            relays_missing_in_scenario = 0
            for relay_name in all_relay_names: # Iterate through the MASTER list
                if relay_name in optimized_settings and optimized_settings[relay_name] is not None:
                    # Relay was successfully optimized in this scenario, use its values
                    final_relay_values_for_scenario[relay_name] = optimized_settings[relay_name]
                else:
                    # Relay was NOT optimized in this scenario (or optimization failed for it)
                    # Assign None (which becomes null in JSON)
                    final_relay_values_for_scenario[relay_name] = {"TDS": None, "pickup": None}
                    if relay_name not in optimized_settings:
                         relays_missing_in_scenario += 1
            # --- END MODIFICATION ---

            if relays_missing_in_scenario > 0:
                logger.info(f"({scenario_id}) {relays_missing_in_scenario}/{len(all_relay_names)} relés no participaron en la optimización; asignados con null.")

            list_entry = {
                "scenario_id": scenario_id,
                "timestamp": current_timestamp,
                "relay_values": final_relay_values_for_scenario # Use the completed dictionary
            }
            output_list.append(list_entry)

        logger.info(f"Formato de lista creado con {len(output_list)} entradas de escenario, cada una conteniendo {len(all_relay_names)} relés.")

        # Save the list
        try:
            output_dir = os.path.dirname(OPTIMIZED_SETTINGS_OUTPUT_PATH)
            if output_dir: os.makedirs(output_dir, exist_ok=True) # Create dir if it doesn't exist
            with open(OPTIMIZED_SETTINGS_OUTPUT_PATH, 'w') as file:
                # Use allow_nan=False to ensure None becomes null, and catch potential NaN/Inf errors
                json.dump(output_list, file, indent=2, allow_nan=False)
            logger.info(f"Archivo con ajustes optimizados (formato lista v4, todos relés garantizados) guardado en: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")
        except TypeError as e:
             logger.error(f"Error de tipo al convertir a JSON (posiblemente NaN/Infinity no manejado): {e}")
             logger.error("Revise la función calculate_operation_time y la lógica de asignación de valores.")
        except Exception as e:
             logger.error(f"Error al guardar el archivo de salida {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")
    else:
        logger.warning("No se procesó ningún escenario o la optimización no produjo resultados. No se guardó ningún archivo.")

    logger.info("--- Script de Optimización Finalizado ---")

2025-04-15 11:36:00,496 - INFO - --- Iniciando Script de Optimización de Ajustes de Relés (v4 - Garantizar Todos los Relés) ---
2025-04-15 11:36:00,584 - INFO - Archivo cargado: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json
2025-04-15 11:36:00,634 - INFO - Lista maestra de relés creada. Total de relés únicos encontrados: 74
2025-04-15 11:36:00,645 - INFO - Datos agrupados por escenario. Pares procesados: 6720, Pares omitidos: 80
2025-04-15 11:36:00,656 - INFO - --- Iniciando optimización para scenario_1 ---
2025-04-15 11:36:00,663 - INFO - (scenario_1) Iter 1/250: OF=137.0168, TMT=-23.9243, MaxNegMT=-0.6159, Pairs_Errors=0
2025-04-15 11:36:00,679 - INFO - (scenario_1) Iter 25/250: OF=24.9059, TMT=-6.1438, MaxNegMT=-0.4327, Pairs_Errors=0
2025-04-15 11:36:00,694 - INFO - (scenario_1) Iter 50/250: OF=13.6124, TMT=-2.8717, MaxNegMT=-0.2277, Pairs_Errors=0
2025-04-15 11:36:00,707 - INFO - (scenario_1) Iter 75/250: OF=15.0055, TM

In [10]:
import json
import numpy as np
import logging
import os
import math
import copy
import random
import string
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple, Set

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- File Paths ---
# !!! PLEASE UPDATE THESE PATHS TO YOUR ACTUAL FILE LOCATIONS !!!
RELAY_PAIRS_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json"
OPTIMIZED_SETTINGS_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values.json" # New output name

# --- Constants ---
K = 0.14
N = 0.02
CTI = 0.2
# Optimization Bounds and Parameters
MIN_TDS = 0.05 # Lower practical limit for TDS
MAX_TDS = 1
MIN_PICKUP = 0.05
# Stricter pickup limit relative to Ishc, inspired by snippet
MAX_PICKUP_FACTOR = 0.7
# MAX_TIME still needed for constraint violations, but not for calculation errors
MAX_TIME = 20.0
MAX_ITERATIONS = 250 # More iterations might be needed
# Convergence Targets
TARGET_TMT = -0.005 # Stricter TMT target again
CONVERGENCE_THRESHOLD_TMT = 0.005 # Smaller margin for TMT
CONVERGENCE_THRESHOLD_MT = -0.01 # Max individual MT allowed
# Objective Function Weights
W_TIME = 0.1 # Further decrease weight of total time
W_MT = 15.0 # Increase weight for miscoordination penalty
W_PICKUP_DIFF = 0.0
# Adjustment Step Sizes & Factors (Inspired by Snippet)
AGGRESSIVE_MT_THRESHOLD = -CTI * 0.75 # Threshold to trigger aggressive adjustments
# Aggressive Steps
AGGRESSIVE_TDS_BACKUP_FACTOR = 1.15 # Increase backup TDS more
AGGRESSIVE_TDS_MAIN_FACTOR = 0.90  # Decrease main TDS more
AGGRESSIVE_PICKUP_BACKUP_FACTOR = 1.05 # Increase backup pickup if TDS maxed out
# Normal Steps (additive for TDS, small mult for pickup)
NORMAL_TDS_BACKUP_ADD = 0.02
NORMAL_TDS_MAIN_SUB = 0.01
NORMAL_PICKUP_BACKUP_FACTOR = 1.01
NORMAL_PICKUP_MAIN_FACTOR = 0.99


# --- Helper Functions ---
def load_json_file(file_path: str) -> Optional[Any]:
    try:
        with open(file_path, 'r') as file: data = json.load(file)
        logger.info(f"Archivo cargado: {file_path}"); return data
    except FileNotFoundError: logger.error(f"No encontrado: {file_path}"); return None
    except json.JSONDecodeError as e: logger.error(f"JSON inválido: {file_path}: {e}"); return None
    except Exception as e: logger.error(f"Error carga {file_path}: {e}"); return None

# MODIFIED: Returns Optional[float] - None indicates calculation error
def calculate_operation_time(I_shc: float, I_pi: float, TDS: float) -> Optional[float]:
    """Calculates relay operation time. Returns None if calculation is impossible."""
    # Basic Input Validation / Conditions for non-calculation
    if I_pi <= 0 or I_shc <= 0 or TDS < MIN_TDS or TDS > MAX_TDS:
        # logger.debug(f"Calc Error: Invalid input I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}")
        return None # Return None for impossible basic inputs

    # Pickup Constraint Check (Violation leads to MAX_TIME, not None)
    if I_pi > I_shc * MAX_PICKUP_FACTOR:
        # logger.debug(f"Calc Constraint Violation: I_pi={I_pi} > I_shc*factor={I_shc * MAX_PICKUP_FACTOR}")
        return MAX_TIME # Return MAX_TIME for pickup constraint violation

    M = I_shc / I_pi
    # Check for M close to or below 1 (prevents division by zero/negative log)
    if M <= 1.0001:
        # logger.debug(f"Calc Error: M={M} <= 1.0001")
        return None # Return None if M is too small

    try:
        denominator = M**N - 1
        # Check for near-zero denominator
        if abs(denominator) < 1e-9:
            # logger.debug(f"Calc Error: Denominator near zero ({denominator})")
            return None # Return None for division by zero

        time = TDS * (K / denominator)

        # Check for non-finite or non-positive results
        if not np.isfinite(time) or time <= 0:
             # logger.debug(f"Calc Error: Result non-finite or non-positive ({time})")
             return None # Return None for invalid results

        return min(time, MAX_TIME) # Return calculated time, capped by MAX_TIME

    except (OverflowError, ValueError) as e:
        # logger.debug(f"Calc Error: Math exception M={M}, N={N} -> {e}")
        return None # Return None on math errors
    except Exception as e:
        logger.error(f"Excepción inesperada en calc_op_time (I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}): {e}")
        return None # Return None for any other unexpected error

def group_data_by_scenario(relay_pairs_data: List[Dict]) -> Dict[str, Dict[str, Any]]:
    scenario_map: Dict[str, Dict[str, Any]] = {}
    processed_pairs_count = 0
    skipped_pairs_count = 0
    for i, pair_entry in enumerate(relay_pairs_data):
        scenario_id = pair_entry.get("scenario_id")
        main_relay_info = pair_entry.get('main_relay'); backup_relay_info = pair_entry.get('backup_relay')
        if not scenario_id or not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            # logger.warning(f"Omitiendo entrada de par {i+1}: Falta scenario_id o info de relé inválida.")
            skipped_pairs_count+=1; continue
        if scenario_id not in scenario_map: scenario_map[scenario_id] = {"pairs_info": [], "initial_settings": {}, "relays": set()}
        main_relay = main_relay_info.get('relay'); backup_relay = backup_relay_info.get('relay')
        I_shc_main = main_relay_info.get('Ishc'); I_shc_backup = backup_relay_info.get('Ishc')
        # Ensure Ishc are positive numbers for calculations
        if not (main_relay and backup_relay and
                isinstance(I_shc_main, (int, float)) and I_shc_main > 0 and
                isinstance(I_shc_backup, (int, float)) and I_shc_backup > 0):
            # logger.warning(f"Omitiendo par {main_relay}/{backup_relay} en {scenario_id}: Falta nombre de relé o Ishc inválido/no positivo (Main: {I_shc_main}, Backup: {I_shc_backup}).")
            skipped_pairs_count+=1; continue

        processed_pairs_count += 1
        scenario_map[scenario_id]['relays'].add(main_relay); scenario_map[scenario_id]['relays'].add(backup_relay)
        initial_settings_scenario = scenario_map[scenario_id]['initial_settings']
        for r_name, r_info in [(main_relay, main_relay_info), (backup_relay, backup_relay_info)]:
            if r_name not in initial_settings_scenario:
                tds = r_info.get('TDS'); pickup = r_info.get('pick_up') # Key is 'pick_up' in input
                if isinstance(tds, (int, float)) and isinstance(pickup, (int, float)):
                    initial_settings_scenario[r_name] = {
                        'TDS_initial': float(tds),
                        'pickup_initial': float(pickup)
                         }
                # else:
                    # logger.warning(f"({scenario_id}) Config inicial TDS/pickup no encontrada o inválida para '{r_name}' en la entrada de par. Se usarán valores por defecto si es necesario más adelante.")
        scenario_map[scenario_id]['pairs_info'].append({"main_relay": main_relay, "backup_relay": backup_relay, "I_shc_main": float(I_shc_main), "I_shc_backup": float(I_shc_backup)})

    logger.info(f"Datos agrupados por escenario. Pares procesados: {processed_pairs_count}, Pares omitidos: {skipped_pairs_count}")
    return scenario_map


# --- Scenario-Specific Optimization Function (MODIFIED for None handling) ---
def run_scenario_optimization(
    scenario_id: str,
    pairs_info: List[Dict],
    initial_settings: Dict[str, Dict[str, float]], # Contains TDS_initial, pickup_initial
    relays_in_scenario: Set[str]
) -> Dict[str, Dict[str, float]]:
    """Performs iterative optimization, handles calculation errors (None), excludes errors from TMT."""

    logger.info(f"--- Iniciando optimización para {scenario_id} ---")
    if not pairs_info: logger.warning(f"({scenario_id}) No hay pares válidos."); return {}

    # 1. Initialize settings - Use initial Pickup, MIN TDS
    relay_settings = {}
    default_pickup = MIN_PICKUP * 1.5 # Start slightly above min

    for relay in relays_in_scenario:
         if relay in initial_settings:
             relay_settings[relay] = {
                 "TDS": MIN_TDS,
                 "pickup": max(MIN_PICKUP, initial_settings[relay]['pickup_initial'])
             }
         else:
             logger.warning(f"({scenario_id}) Config inicial (pickup) no encontrada para '{relay}'. Usando defecto.")
             relay_settings[relay] = {"TDS": MIN_TDS, "pickup": default_pickup}

    # --- Optimization Loop ---
    last_tmt = float('inf')
    no_improvement_streak = 0

    for iteration in range(MAX_ITERATIONS):
        total_main_time = 0.0; tmt = 0.0; miscoordination_penalty = 0.0
        current_pair_results = []; max_neg_mt = 0
        error_pairs_count = 0 # Count pairs with calculation errors

        # Calculate performance
        for pair in pairs_info:
            main_relay = pair["main_relay"]; backup_relay = pair["backup_relay"]
            I_shc_main = pair["I_shc_main"]; I_shc_backup = pair["I_shc_backup"]
            if main_relay not in relay_settings or backup_relay not in relay_settings: continue # Should not happen with grouping logic

            tds_main = relay_settings[main_relay]["TDS"]
            pickup_main = relay_settings[main_relay]["pickup"]
            tds_backup = relay_settings[backup_relay]["TDS"]
            pickup_backup = relay_settings[backup_relay]["pickup"]

            # Apply pickup bounds BEFORE calculating time
            # Note: MAX_PICKUP_FACTOR constraint is checked *inside* calculate_operation_time now
            pickup_main_bounded = max(MIN_PICKUP, pickup_main)
            pickup_backup_bounded = max(MIN_PICKUP, pickup_backup)

            # Calculate times - MODIFIED: handle None return
            main_time = calculate_operation_time(I_shc_main, pickup_main_bounded, tds_main)
            backup_time = calculate_operation_time(I_shc_backup, pickup_backup_bounded, tds_backup)

            delta_t: Optional[float] = None
            mt: Optional[float] = None

            # Check for calculation errors (None) first
            if main_time is None or backup_time is None:
                delta_t = None
                mt = None
                error_pairs_count += 1
                # logger.debug(f"({scenario_id}) Iter {iteration+1}: Error calc pair {main_relay}({main_time})/{backup_relay}({backup_time})")
            # Check for MAX_TIME (constraint violation) next
            elif main_time >= MAX_TIME or backup_time >= MAX_TIME:
                 delta_t = MAX_TIME # Indicate a large difference due to non-operation
                 mt = MAX_TIME # Large penalty
            # Otherwise, calculate normally
            else:
                 delta_t = backup_time - main_time
                 mt = delta_t - CTI

            current_pair_results.append({
                "main_time": main_time, "backup_time": backup_time, # Can be None, float, or MAX_TIME
                "delta_t": delta_t, "mt": mt, # Can be None, float, or MAX_TIME
                "main_relay": main_relay, "backup_relay": backup_relay,
                "I_shc_main": I_shc_main, "I_shc_backup": I_shc_backup
            })

            # --- Accumulate performance metrics - MODIFIED: Skip None values for TMT/Penalty ---
            # Add main time only if valid and finite
            if main_time is not None and main_time < MAX_TIME:
                total_main_time += main_time

            # Add to TMT, penalty, and track max_neg_mt ONLY if mt is a valid negative number
            if mt is not None and mt < 0:
                tmt += mt
                miscoordination_penalty += mt**2
                max_neg_mt = min(max_neg_mt, mt)
            # --- End Accumulation Modification ---

        # Objective function calculation remains the same structure
        of = W_MT * miscoordination_penalty + W_TIME * total_main_time

        if (iteration + 1) % 25 == 0 or iteration == 0 or tmt > TARGET_TMT*1.1:
             logger.info(f"({scenario_id}) Iter {iteration+1}/{MAX_ITERATIONS}: OF={of:.4f}, TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")

        # Check Convergence (based on valid mt values)
        if tmt >= TARGET_TMT and max_neg_mt >= CONVERGENCE_THRESHOLD_MT:
             logger.info(f"({scenario_id}) Convergencia alcanzada (TMT y MT individual dentro de umbrales).")
             break

        # --- AJUSTE V3 (MODIFIED: Skip pairs with mt=None) ---
        adjustments_made = False
        next_relay_settings = copy.deepcopy(relay_settings)

        for pair_res in current_pair_results:
            # --- ADDED: Skip adjustment if mt could not be calculated ---
            if pair_res["mt"] is None:
                continue
            # --- End Skip ---

            # Adjust only miscoordinated pairs (mt < 0)
            # Note: Pairs with mt=MAX_TIME (due to MAX_TIME op times) will not trigger adjustment here.
            # This might be desired (don't adjust based on non-operating pairs) or might need refinement
            # if MAX_TIME indicates a specific type of coordination failure to fix.
            # Current logic focuses on mt < 0 based on calculable, finite times.
            if pair_res["mt"] < 0:
                main_relay = pair_res["main_relay"]; backup_relay = pair_res["backup_relay"]
                I_shc_main = pair_res["I_shc_main"]; I_shc_backup = pair_res["I_shc_backup"]
                mt_val = pair_res["mt"] # We know mt_val is a valid float here

                tds_main_curr = next_relay_settings[main_relay]["TDS"]
                tds_backup_curr = next_relay_settings[backup_relay]["TDS"]
                pickup_main_curr = next_relay_settings[main_relay]["pickup"]
                pickup_backup_curr = next_relay_settings[backup_relay]["pickup"]

                new_tds_backup = tds_backup_curr
                new_pickup_backup = pickup_backup_curr
                new_tds_main = tds_main_curr
                new_pickup_main = pickup_main_curr

                # Apply adjustments based on severity
                if mt_val < AGGRESSIVE_MT_THRESHOLD:
                    # Aggressive adjustment
                    logger.debug(f"({scenario_id}) Iter {iteration+1}: Ajuste Agresivo para {main_relay}/{backup_relay} (mt={mt_val:.4f})")
                    new_tds_backup = min(MAX_TDS, tds_backup_curr * AGGRESSIVE_TDS_BACKUP_FACTOR)
                    if abs(new_tds_backup - MAX_TDS) < 1e-6:
                        new_pickup_backup = min(I_shc_backup * MAX_PICKUP_FACTOR, max(MIN_PICKUP, pickup_backup_curr * AGGRESSIVE_PICKUP_BACKUP_FACTOR))
                    new_tds_main = max(MIN_TDS, tds_main_curr * AGGRESSIVE_TDS_MAIN_FACTOR)
                    # Optional main pickup decrease (apply bounds)
                    # new_pickup_main = min(I_shc_main * MAX_PICKUP_FACTOR, max(MIN_PICKUP, pickup_main_curr * 0.99))

                else:
                    # Normal adjustment
                    logger.debug(f"({scenario_id}) Iter {iteration+1}: Ajuste Normal para {main_relay}/{backup_relay} (mt={mt_val:.4f})")
                    new_tds_backup = min(MAX_TDS, max(MIN_TDS, tds_backup_curr + NORMAL_TDS_BACKUP_ADD))
                    new_tds_main = min(MAX_TDS, max(MIN_TDS, tds_main_curr - NORMAL_TDS_MAIN_SUB))
                    # Optional pickup adjustments (apply bounds)
                    # new_pickup_backup = min(I_shc_backup * MAX_PICKUP_FACTOR, max(MIN_PICKUP, pickup_backup_curr * NORMAL_PICKUP_BACKUP_FACTOR))
                    # new_pickup_main = min(I_shc_main * MAX_PICKUP_FACTOR, max(MIN_PICKUP, pickup_main_curr * NORMAL_PICKUP_MAIN_FACTOR))

                # Update the temporary settings dict
                next_relay_settings[backup_relay]["TDS"] = new_tds_backup
                next_relay_settings[main_relay]["TDS"] = new_tds_main
                next_relay_settings[backup_relay]["pickup"] = new_pickup_backup
                next_relay_settings[main_relay]["pickup"] = new_pickup_main

                adjustments_made = True
        # --- End Adjustment Modification ---

        if adjustments_made:
            relay_settings = next_relay_settings
        elif iteration > 15: # Stop if no adjustments needed after initial phase
             logger.info(f"({scenario_id}) No se realizaron ajustes en iteración {iteration + 1}. Deteniendo.")
             break

        # Check for stagnation (based on TMT from valid pairs)
        if abs(tmt - last_tmt) < 0.0001:
            no_improvement_streak += 1
        else:
            no_improvement_streak = 0
        last_tmt = tmt

        if no_improvement_streak >= 25:
            logger.warning(f"({scenario_id}) TMT no mejora significativamente ({no_improvement_streak} iteraciones). Deteniendo.")
            break

    else: # Executes if loop finishes without break
        logger.warning(f"({scenario_id}) La optimización no convergió (o no mejoró) después de {MAX_ITERATIONS} iteraciones.")

    # Format final results
    formatted_settings = {}
    relays_max_ishc = {} # Cache max Ishc per relay for final bounding
    for relay in relays_in_scenario:
        max_ishc = 0
        for p in pairs_info:
             if p['main_relay'] == relay: max_ishc = max(max_ishc, p.get('I_shc_main', 0))
             if p['backup_relay'] == relay: max_ishc = max(max_ishc, p.get('I_shc_backup', 0))
        relays_max_ishc[relay] = max_ishc if max_ishc > 0 else (default_pickup / MIN_PICKUP) # Avoid Ishc=0

    for relay, settings in relay_settings.items():
        final_pickup = settings['pickup']
        max_ishc_relay = relays_max_ishc.get(relay, default_pickup / MIN_PICKUP)

        # Apply final bounds using the max Ishc seen by this relay in this scenario
        final_pickup_bounded = min(max_ishc_relay * MAX_PICKUP_FACTOR, max(MIN_PICKUP, final_pickup))
        final_tds_bounded = min(MAX_TDS, max(MIN_TDS, settings['TDS']))

        formatted_settings[relay] = {
            "TDS": float(f"{final_tds_bounded:.5f}"),
            "pickup": float(f"{final_pickup_bounded:.5f}")
        }
    logger.info(f"--- Optimización finalizada para {scenario_id} ---")
    return formatted_settings


# --- Script Execution ---
if __name__ == "__main__":
    logger.info("--- Iniciando Script de Optimización de Ajustes de Relés (v3 - Null Handling) ---")
    # !!! ENSURE PATHS ARE CORRECT !!!
    if not os.path.exists(RELAY_PAIRS_PATH):
         logger.error(f"¡Error Crítico! No se encontró el archivo de entrada: {RELAY_PAIRS_PATH}")
         raise SystemExit("Archivo de pares de relés no encontrado.")

    relay_pairs_data = load_json_file(RELAY_PAIRS_PATH)
    if relay_pairs_data is None: raise SystemExit("Error crítico al cargar archivo de pares.")

    scenario_data_map = group_data_by_scenario(relay_pairs_data)
    if not scenario_data_map: raise SystemExit("Error crítico al procesar datos de entrada. No se encontraron escenarios o pares válidos.")

    all_optimized_settings = {}
    for scenario_id, scenario_data in scenario_data_map.items():
        if not scenario_data.get('relays') or not scenario_data.get('pairs_info'):
             logger.warning(f"Omitiendo escenario '{scenario_id}': No contiene relés o pares de información válidos después del procesamiento inicial.")
             continue
        optimized_settings_for_scenario = run_scenario_optimization(
            scenario_id,
            scenario_data['pairs_info'],
            scenario_data['initial_settings'],
            scenario_data['relays']
        )
        if optimized_settings_for_scenario:
             all_optimized_settings[scenario_id] = optimized_settings_for_scenario
        else:
             logger.warning(f"La optimización no produjo resultados para el escenario: {scenario_id}")


    # Format output as list
    output_list = []
    if all_optimized_settings:
        logger.info("Formateando resultados optimizados en la estructura de lista deseada...")
        for scenario_id, optimized_settings in all_optimized_settings.items():
            # Use a consistent way to generate OID-like strings if needed for DB
            # generated_oid = ''.join(random.choices(string.hexdigits.lower(), k=24))
            # Or just use scenario_id and timestamp for uniqueness if OID isn't strictly required
            current_timestamp = datetime.now(timezone.utc).isoformat(timespec='microseconds').replace('+00:00', 'Z') # Ensure Z format
            list_entry = {
                # "_id": { "$oid": generated_oid }, # Uncomment if MongoDB OID structure is needed
                "scenario_id": scenario_id,
                "timestamp": current_timestamp,
                "relay_values": optimized_settings # Contains optimized {'RelayName': {'TDS': x, 'pickup': y}}
            }
            output_list.append(list_entry)
        logger.info(f"Formato de lista creado con {len(output_list)} escenarios optimizados.")

        # Save the list
        try:
            output_dir = os.path.dirname(OPTIMIZED_SETTINGS_OUTPUT_PATH)
            if output_dir: os.makedirs(output_dir, exist_ok=True)
            with open(OPTIMIZED_SETTINGS_OUTPUT_PATH, 'w') as file:
                json.dump(output_list, file, indent=2, allow_nan=False) # Ensure NaN/Inf are not used, None becomes null
            logger.info(f"Archivo con ajustes optimizados (formato lista v3, null handling) guardado en: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")
        except Exception as e:
             logger.error(f"Error al guardar el archivo de salida {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")
    else:
        logger.error("La optimización falló o no produjo resultados para ningún escenario. No se guardó ningún archivo.")

    logger.info("--- Script de Optimización Finalizado ---")

2025-04-15 01:40:32,699 - INFO - --- Iniciando Script de Optimización de Ajustes de Relés (v3 - Null Handling) ---
2025-04-15 01:40:32,734 - INFO - Archivo cargado: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json
2025-04-15 01:40:32,755 - INFO - Datos agrupados por escenario. Pares procesados: 6720, Pares omitidos: 80
2025-04-15 01:40:32,757 - INFO - --- Iniciando optimización para scenario_1 ---
2025-04-15 01:40:32,758 - INFO - (scenario_1) Iter 1/250: OF=137.0168, TMT=-23.9243, MaxNegMT=-0.6159, Errors=0
2025-04-15 01:40:32,865 - INFO - (scenario_1) Iter 25/250: OF=26.5569, TMT=-6.0374, MaxNegMT=-0.4715, Errors=0
2025-04-15 01:40:32,917 - INFO - (scenario_1) Iter 50/250: OF=12.8501, TMT=-2.8837, MaxNegMT=-0.2159, Errors=0
2025-04-15 01:40:32,963 - INFO - (scenario_1) Iter 75/250: OF=15.3406, TMT=-1.9357, MaxNegMT=-0.3393, Errors=0
2025-04-15 01:40:32,975 - INFO - (scenario_1) Iter 100/250: OF=16.7349, TMT=-1.4697, MaxNegMT=-

## 15 Abril


In [13]:
import json
import numpy as np
import logging
import os
import math
import copy
import random
import string
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple, Set

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- File Paths ---
# !!! PLEASE UPDATE THESE PATHS TO YOUR ACTUAL FILE LOCATIONS !!!
RELAY_PAIRS_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json"
OPTIMIZED_SETTINGS_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values.json" # New output name v4

# --- Constants ---
K = 0.14
N = 0.02
CTI = 0.2
# Optimization Bounds and Parameters
MIN_TDS = 0.05 # Lower practical limit for TDS
MAX_TDS = 1
MIN_PICKUP = 0.05
# Stricter pickup limit relative to Ishc, inspired by snippet
MAX_PICKUP_FACTOR = 0.7
# MAX_TIME still needed for constraint violations, but not for calculation errors
MAX_TIME = 20.0
MAX_ITERATIONS = 250 # More iterations might be needed
# Convergence Targets
TARGET_TMT = -0.005 # Target for *total* miscoordination time (can stay negative)
# ***MODIFIED: New threshold for individual Margin Time (MT)***
MIN_ALLOWED_INDIVIDUAL_MT = -0.009 # Convergence threshold for the most negative individual MT
CONVERGENCE_THRESHOLD_TMT = 0.005 # Smaller margin for TMT (still used for stagnation check)
# Objective Function Weights
W_TIME = 0.1 # Further decrease weight of total time
W_MT = 15.0 # Increase weight for miscoordination penalty
W_PICKUP_DIFF = 0.0
# Adjustment Step Sizes & Factors (Inspired by Snippet)
AGGRESSIVE_MT_THRESHOLD = -CTI * 0.75 # Threshold to trigger aggressive adjustments
# Aggressive Steps
AGGRESSIVE_TDS_BACKUP_FACTOR = 1.15 # Increase backup TDS more
AGGRESSIVE_TDS_MAIN_FACTOR = 0.90  # Decrease main TDS more
AGGRESSIVE_PICKUP_BACKUP_FACTOR = 1.05 # Increase backup pickup if TDS maxed out
# Normal Steps (additive for TDS, small mult for pickup)
NORMAL_TDS_BACKUP_ADD = 0.02
NORMAL_TDS_MAIN_SUB = 0.01
NORMAL_PICKUP_BACKUP_FACTOR = 1.01
NORMAL_PICKUP_MAIN_FACTOR = 0.99


# --- Helper Functions ---
def load_json_file(file_path: str) -> Optional[Any]:
    try:
        with open(file_path, 'r') as file: data = json.load(file)
        logger.info(f"Archivo cargado: {file_path}"); return data
    except FileNotFoundError: logger.error(f"No encontrado: {file_path}"); return None
    except json.JSONDecodeError as e: logger.error(f"JSON inválido: {file_path}: {e}"); return None
    except Exception as e: logger.error(f"Error carga {file_path}: {e}"); return None

# MODIFIED: Returns Optional[float] - None indicates calculation error
def calculate_operation_time(I_shc: float, I_pi: float, TDS: float) -> Optional[float]:
    """Calculates relay operation time. Returns None if calculation is impossible."""
    # Basic Input Validation / Conditions for non-calculation
    if I_pi <= 0 or I_shc <= 0 or TDS < MIN_TDS or TDS > MAX_TDS:
        # logger.debug(f"Calc Error: Invalid input I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}")
        return None # Return None for impossible basic inputs

    # Pickup Constraint Check (Violation leads to MAX_TIME, not None)
    # Allow pickup up to Ishc*factor. If it exceeds, it's a constraint violation.
    if I_pi > I_shc * MAX_PICKUP_FACTOR:
        # logger.debug(f"Calc Constraint Violation: I_pi={I_pi} > I_shc*factor={I_shc * MAX_PICKUP_FACTOR}")
        return MAX_TIME # Return MAX_TIME for pickup constraint violation

    M = I_shc / I_pi
    # Check for M close to or below 1 (prevents division by zero/negative log)
    # If pickup is *valid* but M <= 1, it means the relay shouldn't operate -> MAX_TIME
    if M <= 1.0:
        # logger.debug(f"Non-operation: M={M} <= 1.0 (I_shc={I_shc}, I_pi={I_pi})")
        return MAX_TIME # Return MAX_TIME if M indicates non-operation

    try:
        denominator = M**N - 1
        # Check for near-zero denominator (should be less likely now M > 1 is enforced)
        if abs(denominator) < 1e-9:
            # logger.debug(f"Calc Error: Denominator near zero ({denominator})")
            return None # Return None for division by zero (potential numerical instability)

        time = TDS * (K / denominator)

        # Check for non-finite or non-positive results
        if not np.isfinite(time) or time <= 0:
             # logger.debug(f"Calc Error: Result non-finite or non-positive ({time})")
             return None # Return None for invalid results

        return min(time, MAX_TIME) # Return calculated time, capped by MAX_TIME

    except (OverflowError, ValueError) as e:
        # logger.debug(f"Calc Error: Math exception M={M}, N={N} -> {e}")
        return None # Return None on math errors
    except Exception as e:
        logger.error(f"Excepción inesperada en calc_op_time (I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}): {e}")
        return None # Return None for any other unexpected error

def group_data_by_scenario(relay_pairs_data: List[Dict]) -> Dict[str, Dict[str, Any]]:
    scenario_map: Dict[str, Dict[str, Any]] = {}
    processed_pairs_count = 0
    skipped_pairs_count = 0
    for i, pair_entry in enumerate(relay_pairs_data):
        scenario_id = pair_entry.get("scenario_id")
        main_relay_info = pair_entry.get('main_relay'); backup_relay_info = pair_entry.get('backup_relay')
        if not scenario_id or not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            # logger.warning(f"Omitiendo entrada de par {i+1}: Falta scenario_id o info de relé inválida.")
            skipped_pairs_count+=1; continue
        if scenario_id not in scenario_map: scenario_map[scenario_id] = {"pairs_info": [], "initial_settings": {}, "relays": set()}
        main_relay = main_relay_info.get('relay'); backup_relay = backup_relay_info.get('relay')
        I_shc_main = main_relay_info.get('Ishc'); I_shc_backup = backup_relay_info.get('Ishc')
        # Ensure Ishc are positive numbers for calculations
        if not (main_relay and backup_relay and
                isinstance(I_shc_main, (int, float)) and I_shc_main > 0 and
                isinstance(I_shc_backup, (int, float)) and I_shc_backup > 0):
            # logger.warning(f"Omitiendo par {main_relay}/{backup_relay} en {scenario_id}: Falta nombre de relé o Ishc inválido/no positivo (Main: {I_shc_main}, Backup: {I_shc_backup}).")
            skipped_pairs_count+=1; continue

        processed_pairs_count += 1
        scenario_map[scenario_id]['relays'].add(main_relay); scenario_map[scenario_id]['relays'].add(backup_relay)
        initial_settings_scenario = scenario_map[scenario_id]['initial_settings']
        for r_name, r_info in [(main_relay, main_relay_info), (backup_relay, backup_relay_info)]:
            if r_name not in initial_settings_scenario:
                tds = r_info.get('TDS'); pickup = r_info.get('pick_up') # Key is 'pick_up' in input
                if isinstance(tds, (int, float)) and isinstance(pickup, (int, float)):
                    initial_settings_scenario[r_name] = {
                        'TDS_initial': float(tds),
                        'pickup_initial': float(pickup)
                         }
                # else:
                    # logger.warning(f"({scenario_id}) Config inicial TDS/pickup no encontrada o inválida para '{r_name}' en la entrada de par. Se usarán valores por defecto si es necesario más adelante.")
        scenario_map[scenario_id]['pairs_info'].append({"main_relay": main_relay, "backup_relay": backup_relay, "I_shc_main": float(I_shc_main), "I_shc_backup": float(I_shc_backup)})

    logger.info(f"Datos agrupados por escenario. Pares procesados: {processed_pairs_count}, Pares omitidos: {skipped_pairs_count}")
    return scenario_map


# --- Scenario-Specific Optimization Function (MODIFIED for None handling and new convergence) ---
def run_scenario_optimization(
    scenario_id: str,
    pairs_info: List[Dict],
    initial_settings: Dict[str, Dict[str, float]], # Contains TDS_initial, pickup_initial
    relays_in_scenario: Set[str]
) -> Dict[str, Dict[str, float]]:
    """Performs iterative optimization, handles calculation errors (None), excludes errors from TMT, and uses the new individual MT convergence threshold."""

    logger.info(f"--- Iniciando optimización para {scenario_id} ---")
    if not pairs_info: logger.warning(f"({scenario_id}) No hay pares válidos."); return {}

    # 1. Initialize settings - Use initial Pickup, MIN TDS
    relay_settings = {}
    default_pickup = MIN_PICKUP * 1.5 # Start slightly above min

    for relay in relays_in_scenario:
         if relay in initial_settings:
             relay_settings[relay] = {
                 "TDS": MIN_TDS,
                 "pickup": max(MIN_PICKUP, initial_settings[relay]['pickup_initial'])
             }
         else:
             logger.warning(f"({scenario_id}) Config inicial (pickup) no encontrada para '{relay}'. Usando defecto.")
             relay_settings[relay] = {"TDS": MIN_TDS, "pickup": default_pickup}

    # --- Optimization Loop ---
    last_tmt = float('inf')
    no_improvement_streak = 0

    for iteration in range(MAX_ITERATIONS):
        total_main_time = 0.0; tmt = 0.0; miscoordination_penalty = 0.0
        current_pair_results = []; max_neg_mt = 0 # Initialize to 0 (least negative possible)
        error_pairs_count = 0 # Count pairs with calculation errors

        # Calculate performance
        for pair in pairs_info:
            main_relay = pair["main_relay"]; backup_relay = pair["backup_relay"]
            I_shc_main = pair["I_shc_main"]; I_shc_backup = pair["I_shc_backup"]
            if main_relay not in relay_settings or backup_relay not in relay_settings: continue # Should not happen with grouping logic

            tds_main = relay_settings[main_relay]["TDS"]
            pickup_main = relay_settings[main_relay]["pickup"]
            tds_backup = relay_settings[backup_relay]["TDS"]
            pickup_backup = relay_settings[backup_relay]["pickup"]

            # Apply pickup bounds BEFORE calculating time
            # MAX_PICKUP_FACTOR constraint is checked *inside* calculate_operation_time
            pickup_main_bounded = max(MIN_PICKUP, pickup_main)
            pickup_backup_bounded = max(MIN_PICKUP, pickup_backup)

            # Calculate times - MODIFIED: handle None return
            main_time = calculate_operation_time(I_shc_main, pickup_main_bounded, tds_main)
            backup_time = calculate_operation_time(I_shc_backup, pickup_backup_bounded, tds_backup)

            delta_t: Optional[float] = None
            mt: Optional[float] = None

            # Check for calculation errors (None) first
            if main_time is None or backup_time is None:
                delta_t = None
                mt = None
                error_pairs_count += 1
                # logger.debug(f"({scenario_id}) Iter {iteration+1}: Error calc pair {main_relay}({main_time})/{backup_relay}({backup_time})")
            # Check for MAX_TIME (constraint violation or non-operation) next
            elif main_time >= MAX_TIME or backup_time >= MAX_TIME:
                 # If one is MAX_TIME and the other isn't, coordination is definitely violated.
                 # If both are MAX_TIME, they don't coordinate, but it's not a typical miscoordination.
                 # Assign a large penalty value to mt to indicate a severe issue, but not infinity.
                 # Avoid using MAX_TIME directly for mt calculation if possible to distinguish from calc error
                 delta_t = backup_time - main_time if main_time < MAX_TIME and backup_time < MAX_TIME else MAX_TIME
                 # If delta_t is MAX_TIME (or either time was MAX_TIME), assign a large penalty to mt
                 mt = MAX_TIME * 2 # Assign a large penalty if times are MAX_TIME
                 # logger.debug(f"({scenario_id}) Iter {iteration+1}: MAX_TIME detected for {main_relay}({main_time})/{backup_relay}({backup_time}) -> mt={mt}")
            # Otherwise, calculate normally
            else:
                 delta_t = backup_time - main_time
                 mt = delta_t - CTI

            current_pair_results.append({
                "main_time": main_time, "backup_time": backup_time, # Can be None, float, or MAX_TIME
                "delta_t": delta_t, "mt": mt, # Can be None, float, or large penalty value
                "main_relay": main_relay, "backup_relay": backup_relay,
                "I_shc_main": I_shc_main, "I_shc_backup": I_shc_backup
            })

            # --- Accumulate performance metrics - MODIFIED: Skip None values for TMT/Penalty ---
            # Add main time only if valid and finite
            if main_time is not None and main_time < MAX_TIME:
                total_main_time += main_time

            # Add to TMT, penalty, and track max_neg_mt ONLY if mt is a valid negative number
            # Exclude the large penalty values assigned for MAX_TIME cases from normal TMT/max_neg_mt
            if mt is not None and mt < 0 and mt < MAX_TIME: # Check mt is negative AND not the large penalty
                tmt += mt
                miscoordination_penalty += mt**2
                max_neg_mt = min(max_neg_mt, mt) # Update the most negative MT found so far
            # --- End Accumulation Modification ---

        # Objective function calculation remains the same structure
        of = W_MT * miscoordination_penalty + W_TIME * total_main_time

        if (iteration + 1) % 25 == 0 or iteration == 0 or tmt > TARGET_TMT*1.1 :
             logger.info(f"({scenario_id}) Iter {iteration+1}/{MAX_ITERATIONS}: OF={of:.4f}, TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")

        # --- Check Convergence (MODIFIED: using MIN_ALLOWED_INDIVIDUAL_MT) ---
        # Converge if the total miscoordination is below target AND the worst individual miscoordination is above the minimum allowed limit
        if tmt >= TARGET_TMT and max_neg_mt >= MIN_ALLOWED_INDIVIDUAL_MT:
             logger.info(f"({scenario_id}) Convergencia alcanzada (TMT={tmt:.4f} >= {TARGET_TMT}, MaxNegMT={max_neg_mt:.4f} >= {MIN_ALLOWED_INDIVIDUAL_MT}).")
             break
        # --- End Convergence Check Modification ---

        # --- AJUSTE V3 (MODIFIED: Skip pairs with mt=None or mt indicating MAX_TIME penalty) ---
        adjustments_made = False
        next_relay_settings = copy.deepcopy(relay_settings)

        for pair_res in current_pair_results:
            # --- ADDED: Skip adjustment if mt could not be calculated or indicates MAX_TIME penalty ---
            if pair_res["mt"] is None or pair_res["mt"] >= MAX_TIME :
                continue
            # --- End Skip ---

            # Adjust only miscoordinated pairs (mt < 0 AND mt is not the large penalty value)
            if pair_res["mt"] < 0:
                main_relay = pair_res["main_relay"]; backup_relay = pair_res["backup_relay"]
                I_shc_main = pair_res["I_shc_main"]; I_shc_backup = pair_res["I_shc_backup"]
                mt_val = pair_res["mt"] # We know mt_val is a valid negative float here

                tds_main_curr = next_relay_settings[main_relay]["TDS"]
                tds_backup_curr = next_relay_settings[backup_relay]["TDS"]
                pickup_main_curr = next_relay_settings[main_relay]["pickup"]
                pickup_backup_curr = next_relay_settings[backup_relay]["pickup"]

                new_tds_backup = tds_backup_curr
                new_pickup_backup = pickup_backup_curr
                new_tds_main = tds_main_curr
                new_pickup_main = pickup_main_curr

                # Apply adjustments based on severity
                if mt_val < AGGRESSIVE_MT_THRESHOLD:
                    # Aggressive adjustment
                    # logger.debug(f"({scenario_id}) Iter {iteration+1}: Ajuste Agresivo para {main_relay}/{backup_relay} (mt={mt_val:.4f})")
                    new_tds_backup = min(MAX_TDS, tds_backup_curr * AGGRESSIVE_TDS_BACKUP_FACTOR)
                    # Only adjust pickup if TDS hits max and miscoordination is still severe
                    if abs(new_tds_backup - MAX_TDS) < 1e-6:
                         # Ensure pickup adjustment respects the MAX_PICKUP_FACTOR limit relative to *its* Ishc
                         max_allowed_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
                         new_pickup_backup = min(max_allowed_pickup_backup, max(MIN_PICKUP, pickup_backup_curr * AGGRESSIVE_PICKUP_BACKUP_FACTOR))

                    new_tds_main = max(MIN_TDS, tds_main_curr * AGGRESSIVE_TDS_MAIN_FACTOR)
                    # Aggressively decrease main pickup only if main TDS hits min? (Optional refinement)
                    # if abs(new_tds_main - MIN_TDS) < 1e-6:
                    #    max_allowed_pickup_main = I_shc_main * MAX_PICKUP_FACTOR
                    #    new_pickup_main = min(max_allowed_pickup_main, max(MIN_PICKUP, pickup_main_curr * 0.98)) # Example factor

                else:
                    # Normal adjustment
                    # logger.debug(f"({scenario_id}) Iter {iteration+1}: Ajuste Normal para {main_relay}/{backup_relay} (mt={mt_val:.4f})")
                    new_tds_backup = min(MAX_TDS, max(MIN_TDS, tds_backup_curr + NORMAL_TDS_BACKUP_ADD))
                    new_tds_main = min(MAX_TDS, max(MIN_TDS, tds_main_curr - NORMAL_TDS_MAIN_SUB))
                    # Subtle pickup adjustments (less critical now with MAX_PICKUP_FACTOR check in calc_time)
                    # max_allowed_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
                    # new_pickup_backup = min(max_allowed_pickup_backup, max(MIN_PICKUP, pickup_backup_curr * NORMAL_PICKUP_BACKUP_FACTOR))
                    # max_allowed_pickup_main = I_shc_main * MAX_PICKUP_FACTOR
                    # new_pickup_main = min(max_allowed_pickup_main, max(MIN_PICKUP, pickup_main_curr * NORMAL_PICKUP_MAIN_FACTOR))


                # Update the temporary settings dict, ensuring bounds are respected immediately
                next_relay_settings[backup_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_backup))
                next_relay_settings[main_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_main))
                # Apply pickup bounds during adjustment application
                max_pickup_b = I_shc_backup * MAX_PICKUP_FACTOR
                max_pickup_m = I_shc_main * MAX_PICKUP_FACTOR
                next_relay_settings[backup_relay]["pickup"] = min(max_pickup_b, max(MIN_PICKUP, new_pickup_backup))
                next_relay_settings[main_relay]["pickup"] = min(max_pickup_m, max(MIN_PICKUP, new_pickup_main))


                adjustments_made = True
        # --- End Adjustment Modification ---

        if adjustments_made:
            relay_settings = next_relay_settings # Apply the adjusted settings for the next iteration
        elif iteration > 15: # Stop if no adjustments needed after initial phase
             logger.info(f"({scenario_id}) No se realizaron ajustes en iteración {iteration + 1}. Deteniendo.")
             break

        # Check for stagnation (based on TMT from valid pairs)
        # Use the original CONVERGENCE_THRESHOLD_TMT for stagnation check magnitude
        if abs(tmt - last_tmt) < CONVERGENCE_THRESHOLD_TMT: # Check if TMT improvement is negligible
            no_improvement_streak += 1
        else:
            no_improvement_streak = 0
        last_tmt = tmt

        if no_improvement_streak >= 25:
            logger.warning(f"({scenario_id}) TMT no mejora significativamente ({no_improvement_streak} iteraciones). Deteniendo.")
            break

    else: # Executes if loop finishes without break (max iterations reached)
        logger.warning(f"({scenario_id}) La optimización no convergió (o no mejoró) después de {MAX_ITERATIONS} iteraciones.")
        # Log final state if max iterations reached
        logger.warning(f"({scenario_id}) Estado final: TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")


    # Format final results
    formatted_settings = {}
    relays_max_ishc = {} # Cache max Ishc per relay for final bounding
    for relay in relays_in_scenario:
        max_ishc = 0
        for p in pairs_info:
             if p['main_relay'] == relay: max_ishc = max(max_ishc, p.get('I_shc_main', 0))
             if p['backup_relay'] == relay: max_ishc = max(max_ishc, p.get('I_shc_backup', 0))
        # Ensure there's a sensible Ishc even if a relay only appears with Ishc=0 somehow (use default pickup relation)
        relays_max_ishc[relay] = max_ishc if max_ishc > 0 else (default_pickup / MIN_PICKUP)

    for relay, settings in relay_settings.items():
        final_pickup = settings['pickup']
        max_ishc_relay = relays_max_ishc.get(relay, default_pickup / MIN_PICKUP) # Fallback Ishc

        # Apply final bounds using the max Ishc seen by this relay in this scenario
        # This ensures the final pickup setting is valid w.r.t the highest fault current it needs to handle
        final_pickup_bounded = min(max_ishc_relay * MAX_PICKUP_FACTOR, max(MIN_PICKUP, final_pickup))
        final_tds_bounded = min(MAX_TDS, max(MIN_TDS, settings['TDS']))

        formatted_settings[relay] = {
            "TDS": float(f"{final_tds_bounded:.5f}"),
            "pickup": float(f"{final_pickup_bounded:.5f}")
        }
    logger.info(f"--- Optimización finalizada para {scenario_id} ---")
    return formatted_settings


# --- Script Execution ---
if __name__ == "__main__":
    logger.info("--- Iniciando Script de Optimización de Ajustes de Relés (v4 - MT Threshold -0.009) ---")
    # !!! ENSURE PATHS ARE CORRECT !!!
    if not os.path.exists(RELAY_PAIRS_PATH):
         logger.error(f"¡Error Crítico! No se encontró el archivo de entrada: {RELAY_PAIRS_PATH}")
         raise SystemExit("Archivo de pares de relés no encontrado.")

    relay_pairs_data = load_json_file(RELAY_PAIRS_PATH)
    if relay_pairs_data is None: raise SystemExit("Error crítico al cargar archivo de pares.")

    scenario_data_map = group_data_by_scenario(relay_pairs_data)
    if not scenario_data_map: raise SystemExit("Error crítico al procesar datos de entrada. No se encontraron escenarios o pares válidos.")

    all_optimized_settings = {}
    successful_scenarios = 0
    failed_scenarios = 0
    for scenario_id, scenario_data in scenario_data_map.items():
        if not scenario_data.get('relays') or not scenario_data.get('pairs_info'):
             logger.warning(f"Omitiendo escenario '{scenario_id}': No contiene relés o pares de información válidos después del procesamiento inicial.")
             failed_scenarios += 1
             continue
        try:
            optimized_settings_for_scenario = run_scenario_optimization(
                scenario_id,
                scenario_data['pairs_info'],
                scenario_data['initial_settings'],
                scenario_data['relays']
            )
            if optimized_settings_for_scenario:
                 all_optimized_settings[scenario_id] = optimized_settings_for_scenario
                 successful_scenarios += 1
            else:
                 logger.warning(f"La optimización no produjo resultados para el escenario: {scenario_id}")
                 failed_scenarios += 1
        except Exception as e:
            logger.error(f"Error inesperado durante optimización del escenario {scenario_id}: {e}", exc_info=True)
            failed_scenarios += 1


    # Format output as list
    output_list = []
    if all_optimized_settings:
        logger.info(f"Optimización completada. Escenarios exitosos: {successful_scenarios}, Escenarios fallidos/omitidos: {failed_scenarios}.")
        logger.info("Formateando resultados optimizados en la estructura de lista deseada...")
        for scenario_id, optimized_settings in all_optimized_settings.items():
            # Use a consistent way to generate OID-like strings if needed for DB
            # generated_oid = ''.join(random.choices(string.hexdigits.lower(), k=24))
            # Or just use scenario_id and timestamp for uniqueness if OID isn't strictly required
            current_timestamp = datetime.now(timezone.utc).isoformat(timespec='microseconds').replace('+00:00', 'Z') # Ensure Z format
            list_entry = {
                # "_id": { "$oid": generated_oid }, # Uncomment if MongoDB OID structure is needed
                "scenario_id": scenario_id,
                "timestamp": current_timestamp,
                "relay_values": optimized_settings # Contains optimized {'RelayName': {'TDS': x, 'pickup': y}}
            }
            output_list.append(list_entry)
        logger.info(f"Formato de lista creado con {len(output_list)} escenarios optimizados.")

        # Save the list
        try:
            output_dir = os.path.dirname(OPTIMIZED_SETTINGS_OUTPUT_PATH)
            if output_dir: os.makedirs(output_dir, exist_ok=True)
            # Use allow_nan=False to ensure compatibility with standard JSON (NaN/Infinity are not valid JSON)
            # Python's None will be converted to JSON null
            with open(OPTIMIZED_SETTINGS_OUTPUT_PATH, 'w') as file:
                json.dump(output_list, file, indent=2, allow_nan=False)
            logger.info(f"Archivo con ajustes optimizados (v4, MT>=-0.009, null handling) guardado en: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")
        except Exception as e:
             logger.error(f"Error al guardar el archivo de salida {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")
    else:
        logger.error(f"La optimización falló o no produjo resultados para ningún escenario ({successful_scenarios} exitosos, {failed_scenarios} fallidos/omitidos). No se guardó ningún archivo.")

    logger.info("--- Script de Optimización Finalizado ---")

2025-04-15 18:43:03,215 - INFO - --- Iniciando Script de Optimización de Ajustes de Relés (v4 - MT Threshold -0.009) ---
2025-04-15 18:43:03,479 - INFO - Archivo cargado: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json
2025-04-15 18:43:03,549 - INFO - Datos agrupados por escenario. Pares procesados: 6720, Pares omitidos: 80
2025-04-15 18:43:03,582 - INFO - --- Iniciando optimización para scenario_1 ---
2025-04-15 18:43:03,591 - INFO - (scenario_1) Iter 1/250: OF=136.7907, TMT=-23.9091, MaxNegMT=-0.6159, Errors=0
2025-04-15 18:43:03,602 - INFO - (scenario_1) Iter 25/250: OF=25.8501, TMT=-5.8611, MaxNegMT=-0.4715, Errors=0
2025-04-15 18:43:03,614 - INFO - (scenario_1) Iter 50/250: OF=12.9755, TMT=-3.0423, MaxNegMT=-0.2159, Errors=0
2025-04-15 18:43:03,624 - INFO - (scenario_1) Iter 75/250: OF=13.8426, TMT=-1.8219, MaxNegMT=-0.2588, Errors=0
2025-04-15 18:43:03,637 - INFO - (scenario_1) Iter 100/250: OF=14.4704, TMT=-1.0355, MaxN